## Pair Programming - Regresión logística 6

### Random Forest

---

In [1]:
# Tratamiento de datos
import numpy as np
import pandas as pd
from tqdm import tqdm

# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../archivos/travel_balanceado_sin_dupl.csv', index_col=0)
df.head(2)

,product_name,net_sales,commision_(in_value),continent,cat_age,cat_duration,net_sales_stand,commision_(in_value)_stand,agency_ADM,agency_ART,...,agency_TST,agency_TTW,agency_type_Airlines,agency_type_Travel Agency,distribution_channel_Offline,distribution_channel_Online,gender_F,gender_M,gender_PNS,claim
0,14,21.0,0.0,6,6,5,-0.431903,-0.516562,0,0,...,0,0,0,1,0,1,0,0,1,0
1,9,20.0,0.0,6,6,8,-0.451726,-0.516562,0,0,...,0,0,0,1,0,1,0,0,1,0


In [3]:
df_stand_bal = df.drop(['net_sales', 'commision_(in_value)'], axis= 1)

**Objetivos**:
### 1. Ajustad un modelo de Random Forest a nuestros datos.

In [4]:
X = df_stand_bal.drop("claim", axis = 1)
y = df_stand_bal["claim"]

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
param = {"max_depth": [4, 6, 8], 
        "max_features": [1,2,3,4],
        "min_samples_split": [30, 50, 100],
        "min_samples_leaf": [30,50,100]} 

In [7]:
gs_rf = GridSearchCV(
            estimator=RandomForestClassifier(random_state=42), 
            param_grid= param, 
            cv=10, 
            verbose=-1) 
        

In [8]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [4, 6, 8], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [30, 50, 100],
                         'min_samples_split': [30, 50, 100]},
             verbose=-1)

In [9]:
bosque = gs_rf.best_estimator_
bosque

RandomForestClassifier(max_depth=8, max_features=4, min_samples_leaf=30,
                       min_samples_split=30, random_state=42)

Prescindimos de sacar visualmente todos los árboles por rendimiento del ordenador.

In [10]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

### 2. Calculad las métricas a nuestro nuevo modelo.

In [11]:
def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    
    # para el test
    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)

    # para el train
    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    

    
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train], 
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train], 
                       "f1": [f1_test, f1_train],
                       "kappa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    
    df["modelo"] = modelo
    return df

In [12]:
dt_results = metricas(y_test, y_pred_test_rf,y_train,  y_pred_train_rf, "Random Forest")
dt_results

,accuracy,precision,recall,f1,kappa,set,modelo
0,0.756571,0.746099,0.638350,0.688031,0.490424,test,Random Forest
1,0.752488,0.747720,0.642847,0.691329,0.486702,train,Random Forest


Vemos en las métricas que el kappa

### 3. Comparad las métricas con los modelos hechos hasta ahora. ¿Cuál es mejor?

In [13]:
resultados_anteriores = pd.read_csv("../archivos/resultados_travel_log+DC.csv", index_col=0)

In [14]:
resultados_todo = pd.concat([resultados_anteriores, dt_results  ], axis = 0)
resultados_todo

,accuracy,precision,recall,f1,kappa,set,modelo
0,0.729013,0.731073,0.562500,0.635802,0.425813,test,Regresión logistica Stan_Bal
1,0.733031,0.749612,0.571830,0.648762,0.439668,train,Regresión logistica Stan_Bal
0,0.985801,0.000000,0.000000,0.000000,-0.000180,test,Regresión logistica Sin Stan-Bal
1,0.982635,0.000000,0.000000,0.000000,-0.000135,train,Regresión logistica Sin Stan-Bal
0,0.728502,0.730284,0.561893,0.635117,0.424732,test,Regresión logistica Sin Stan Con Bal
1,0.732712,0.749660,0.570647,0.648017,0.438885,train,Regresión logistica Sin Stan Con Bal
0,0.985801,0.000000,0.000000,0.000000,-0.000180,test,Regresión logistica Con Stan Sin Bal
1,0.982635,0.000000,0.000000,0.000000,-0.000135,train,Regresión logistica Con Stan Sin Bal
0,0.895892,0.879437,0.871966,0.875686,0.786136,test,Decission Tree Stand_Bal I
1,0.975440,1.000000,0.943039,0.970685,0.949584,train,Decission Tree Stand_Bal I


### ➡️ **_Conclusiones_**

De todos los modelos que tenemos llegamos a la conclusión de que el mejor para experimentar sería el del Decision Tree ya que auqnue tiene un poco de overfitting su precision y recall son muy aceptables, teniendo en cuenta que su kappa es bastante bueno con un 0.77. 

De cara a seguir mejorando podráimos:

- Volver con este modelo, mencionado arriba, y experimentar con sus hiperparámetros para ver si podemos eliminar de alguna forma ese desnivel entre el train y el test.

- U otra opción sería volver más atrás aún y en la fase de balanceo probar otros métodos para igualar las categorías de la variable respuesta: 
    - Podríamos probar a bajar aún más el párametro sampling_strategy del metodo SMOTETomek.
    - O hacer un pequeño downsampling de la categoría mayoritaria y a la vez hacer un pequeño balance con el método SMOTETomek.
